In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-18 22:03:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-04-18 22:03:47 (1.68 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



**review_date column is string type. needs to be converted to date format using to_date function**

In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.count()

3514942

In [9]:
# Read in the Review dataset as a DataFrame
# transform 'review_date' column to a date format/datatype with to_date. to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_df. show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

In [10]:
from pyspark.sql.functions import col
# small chunk of data to test postgres load
review_test = review_df.filter(col("review_id")=="RAB23OVFNCXZQ")
review_test.show()

+-------------+-----------+----------+--------------+-----------+
|    review_id|customer_id|product_id|product_parent|review_date|
+-------------+-----------+----------+--------------+-----------+
|RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
+-------------+-----------+----------+--------------+-----------+



In [11]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count") # alternate method customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28258386|             3|
|   18634862|             3|
|    2072312|             3|
|   33132797|            15|
|   14539137|             1|
|   14552054|             5|
|    9347763|             1|
|   43626894|             2|
|   27314089|            47|
|   36829268|             1|
|   24540309|             2|
|    8501205|             1|
|    4361670|             2|
|   30416085|             2|
|    8306731|             1|
|   11491871|             1|
|   46406103|             1|
|   18417791|             1|
|   51299913|             1|
|   20587872|             4|
+-----------+--------------+
only showing top 20 rows



In [12]:
# small chunk of data to test postgres load
customer_test = customers_df.filter(col("customer_id")=="28258386")
customer_test.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28258386|             3|
+-----------+--------------+



In [13]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()

products_df.show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                             |
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|B002OKKCO8|06-09 KAWASAKI ZX14: GPR V4 Steering Stabilizer (BLACK)                                                                                                   |
|B00OZXP3BY|Oem Factory Stock Genuine 2011 2012 2013 2014 Ford F-150 F150 SuperCrew Without Subwoofer Black Ebony Rubber All Weather Floor Mats Set 3-pc Front & Rear |
|B00HA26HZS|Suzuki Hayabusa GSX1300R Carbon Fiber Motorcycle Tank Protector Pad                                                                                 

In [14]:
# small chunk of data to test postgres load
product_test = products_df.filter(col("product_id")=="B0022ZYS2C")
product_test.show(truncate=False)

+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [17]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [18]:
# small chunk of data to test postgres load
vine_test= vine_df.filter(col("review_id")=="RAB23OVFNCXZQ")
vine_test.show()

+-------------+-----------+-------------+-----------+----+-----------------+
|    review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------------+-----------+-------------+-----------+----+-----------------+
|RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
+-------------+-----------+-------------+-----------+----+-----------------+



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [19]:
# Store environmental variable for PW, user input required
from getpass import getpass
password = getpass('Enter DB Password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://modchallenge.cednbrbvnjbb.us-east-2.rds.amazonaws.com:5432/classdb"
config = {"user":"root", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter DB Password··········


In [20]:
# Write review_id_df to table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS

products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [22]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [37]:
# creat csv of vine_df table in Colab folder to be downloaded
#path = "Users/shama/Documents/UFT BootCamp/Module 16-BigData & Spark/"
vine_df.write.option("header",True).csv("vinecsv")